In [1]:
import pandas as pd
import numpy as np

### utils

In [10]:
def pre_process():
    """
    Pré-processa os dados seletiva_nivel, cruza com os ids de locais.xlsx, normaliza o nome dos circuitos
    e agrupa os eventos em clusters de 5 minutos (info_adicional e local_id)

    """
    
    df1 = pd.read_excel("../data/raw/seletiva_nivel.xlsx", sheet_name='Result 1')
    df2 = pd.read_excel("../data/raw/seletiva_nivel.xlsx", sheet_name='Result 2')
    seletiva_df = pd.concat([df1, df2], ignore_index=True)

    locais_path = "../data/raw/locais.xlsx"
    locais_df = pd.read_excel(locais_path)

    seletiva_df = seletiva_df.drop(columns=['produto_id'])
    seletiva_df = seletiva_df.drop(columns=['local_uuid_id'])#Remover as colunas que não interessam
    seletiva_df = seletiva_df[seletiva_df['local_id'] != 6394] # Este ponto é removido pois ao criar as rotas com o openrouteservice dá erro devido às
                                                               # suas coordenadas

    locais_df.rename(columns={'id': 'local_id'}, inplace=True)

    # Filtrar apenas os IDs comuns
    ids_comuns = set(locais_df['local_id']).intersection(set(seletiva_df['local_id']))
    seletiva_df = seletiva_df[seletiva_df['local_id'].isin(ids_comuns)]

    substituicoes = {
    'Circuito 03 32': 'Circuito 03',
    'Cricuito 08': 'Circuito 08',
    'Circ. Ilhas S': 'Circ. Ilhas',
    'circuito 05': 'Circuito 05',
    'Circ. Ilhas 04': 'Circ. Ilhas',
    'Circuito 09 92': 'Circuito 09',
    'Circuito Ilhas': 'Circ. Ilhas'
    }

    # Ignorar linhas com circuitos "via recolha" ou "GP 91"
    seletiva_df = seletiva_df[~seletiva_df['info_adicional'].str.lower().isin(['via recolha', 'gp 91'])] # Remove-se o gp 91 pois só tem 1 ponto
                                                                                                         # e não há correspondencia a outro circuito
    # Aplicar substituições
    seletiva_df['info_adicional'] = seletiva_df['info_adicional'].replace(substituicoes)

    # 3️⃣ Conversão da coluna 'time' para timestamp
    seletiva_df['timestamp'] = pd.to_datetime(seletiva_df['time'], format='mixed')

    # 4️⃣ Definição do threshold de 5 minutos
    threshold = pd.Timedelta(minutes=5)

    # 5️⃣ Ordenação para garantir que .diff() funcione bem
    df = seletiva_df.sort_values(by=['info_adicional', 'local_id', 'timestamp']).reset_index(drop=True)

    # 7️⃣ Aplicação da clusterização por (info_adicional, local_id)
    df = df.groupby(['info_adicional', 'local_id'], group_keys=False).apply(cluster_sessions)
    
    # 8️⃣ Criação da coluna identificadora de cluster
    df['circuito_local_cluster'] = df['info_adicional'] + '-' + df['local_id'].astype(str) + '-' + df['cluster'].astype(str)

    # 9️⃣ Criação do identificador numérico de cluster
    df['circuito_local_giro_id'] = df['circuito_local_cluster'].rank(method='dense')
    return df
    

In [11]:
def cluster_sessions(group):
   """
    Agrupa eventos de um mesmo grupo (info_adicional e local_id) em clusters de 5 minutos.
    Um novo cluster é iniciado sempre que o intervalo entre dois eventos consecutivos
    for maior que 5 minutos.
    """

    
    threshold = pd.Timedelta(minutes=5)
    group = group.copy()
    group['time_diff'] = group['timestamp'].diff()
    group['cluster'] = (group['time_diff'] > threshold).cumsum()
    return group


In [12]:
def processar_grupo(grupo):
     """
    Processa um grupo de registos (já agrupado por circuito, local e cluster)
    para calcular os níveis médios por tipo e identificar se houve recolha.

    Para cada tipo ('Papel', 'Embalagens', 'Vidro'):
    - Calcula a média dos níveis se houver mais do que uma observacao para o mesmo tipo dentro do cluster
    - Define se houve recolha (quando esta o nivel esta a 0).
    """

    
    tipos = ['Papel', 'Embalagens', 'Vidro']
    niveis = {}
    recolhidos = {}
    
    for tipo in tipos:
        valores = grupo[grupo['tipo'] == tipo]['nivel'].tolist()
        valores_validos = [v for v in valores if pd.notna(v)]

        # Nível
        if not valores_validos:
            niveis[tipo] = np.nan
        else:
            valores_sem_0 = [v for v in valores_validos if v != 0]
            if valores_sem_0:
                niveis[tipo] = np.mean(valores_sem_0)  # usar média aqui
            else:
                niveis[tipo] = 0

        # Recolhido = 1 se algum nivel == 0, senão 0
        recolhidos[tipo] = 1 if 0 in valores_validos else 0

    hora = grupo['Hora'].iloc[0]

    return pd.Series({
        'Data do circuito': grupo['Data do circuito'].iloc[0],
        'Hora': hora,
        'Circuito': grupo['info_adicional'].iloc[0],
        'local_id': grupo['local_id'].iloc[0],
        'cluster': grupo['cluster'].iloc[0],   # <== AQUI adiciona a coluna cluster!
        'Papel': niveis['Papel'],
        'Embalagens': niveis['Embalagens'],
        'Vidro': niveis['Vidro'],
        'Recolhido Papel': recolhidos['Papel'],
        'Recolhido Embalagens': recolhidos['Embalagens'],
        'Recolhido Vidro': recolhidos['Vidro'],
    })


In [13]:
def media_sem_zeros(serie):
    """
    Calcula a média ignorando quando o nivel esta a 0 
    """
    valores = serie[serie > 0]  # só pega valores > 0
    return valores.mean() if not valores.empty else 0  # se não houver valores > 0, retorna 0


In [20]:
def met_1(df_copia):
     """
    Processa o df_copia e transforma em um resumo por cluster.

    Etapas:
     Ordena os dados e agrupa em clusters por circuito/local usando a função cluster_sessions.
     Extrai a data e hora de cada evento.
     Agrega os dados por cluster usando a função processar_grupo.
     Ordena as colunas
    """

    
    df_copia['timestamp'] = pd.to_datetime(df_copia['time'])

    df_copia = df_copia.sort_values(by=['info_adicional', 'local_id', 'timestamp']).reset_index(drop=True)
    threshold = pd.Timedelta(minutes=5)
    
    df_copia = df_copia.groupby(['info_adicional', 'local_id'], group_keys=False).apply(cluster_sessions)
    
    # --- 2. Colunas auxiliares ---
    df_copia['Data do circuito'] = df_copia['timestamp'].dt.date
    df_copia['Hora'] = df_copia['timestamp'].dt.time
    
    # --- 4. Aplicar função por cluster ---
    df_cluster_info = df_copia.groupby(['info_adicional', 'local_id', 'Data do circuito', 'cluster'], group_keys=False).apply(processar_grupo).reset_index(drop=True)
    
    # --- 5. Ordenar para visualização ---
    df_cluster_info = df_cluster_info.sort_values(by=['Data do circuito', 'Hora', 'Circuito', 'local_id', 'cluster']).reset_index(drop=True)

    ##Arredondar
    df_cluster_info[['Papel', 'Embalagens', 'Vidro']] = df_cluster_info[['Papel', 'Embalagens', 'Vidro']].round(0).astype('Int64')

    # Colocar NaN nas colunas em branco
    for coluna in ['Papel', 'Embalagens', 'Vidro']:
        df_cluster_info[coluna] = df_cluster_info[coluna].apply(lambda x: 'NaN' if pd.isna(x) or x == 0 else x) 

        # ORDENAR por Data, Circuito e Hora
    df_cluster_info['Data do circuito'] = pd.to_datetime(df_cluster_info['Data do circuito'])
    df_cluster_info['Hora'] = pd.to_datetime(df_cluster_info['Hora'], format='%H:%M:%S').dt.time
    df_cluster_info = df_cluster_info.sort_values(by=['Data do circuito', 'Circuito', 'Hora']).reset_index(drop=True)
    df_cluster_info['Data do circuito'] = df_cluster_info['Data do circuito'].dt.date  

    return df_cluster_info

In [15]:
df=pre_process()

C:\Users\simao\AppData\Local\Temp\ipykernel_20204\3494139500.py:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['info_adicional', 'local_id'], group_keys=False).apply(cluster_sessions)


### Geral

In [16]:
##FAZER PARA O ANO TODO

In [17]:
df_copia=df.copy()

In [18]:
df_copia['day'] = pd.to_datetime(df_copia['time']).dt.date

In [21]:
df_cluster_info = met_1(df_copia)

C:\Users\simao\AppData\Local\Temp\ipykernel_20204\1189917085.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_copia = df_copia.groupby(['info_adicional', 'local_id'], group_keys=False).apply(cluster_sessions)
C:\Users\simao\AppData\Local\Temp\ipykernel_20204\1189917085.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_cluster_info = df_copia.groupby(['info_adicional', 'local_id', 'Data do circui

In [22]:
df_cluster_info

,Data do circuito,Hora,Circuito,local_id,cluster,Papel,Embalagens,Vidro,Recolhido Papel,Recolhido Embalagens,Recolhido Vidro
0,2024-01-02,04:20:43,Circ. Ilhas,3863,0,NaN,1.0,1.0,0,0,0
1,2024-01-02,04:20:49,Circ. Ilhas,3864,0,NaN,1.0,1.0,0,0,0
2,2024-01-02,04:32:14,Circ. Ilhas,3844,0,2.0,2.0,2.0,0,0,0
3,2024-01-02,04:32:30,Circ. Ilhas,3865,0,4.0,3.0,3.0,1,0,0
4,2024-01-02,04:53:56,Circ. Ilhas,3732,0,NaN,1.0,1.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
165898,2024-12-31,15:01:32,Circuito 12,4294,183,NaN,5.0,3.0,0,1,0
165899,2024-12-31,15:05:56,Circuito 12,4396,186,NaN,5.0,4.0,0,1,0
165900,2024-12-31,15:06:15,Circuito 12,4299,179,NaN,4.0,1.0,0,1,1
165901,2024-12-31,15:11:15,Circuito 12,4296,186,3.0,4.0,4.0,0,1,0


In [23]:
output_path='../data/tabela_com_niveis_recolhas.csv'
df_cluster_info.to_csv(output_path, index=False)
print(f"Ficheiro guardado")

Ficheiro guardado
